In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, timezone
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import os
from dotenv import load_dotenv
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import locale

In [4]:
# Villes françaises avec coordonnées
cities = ['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 
 'Strasbourg', 'Bordeaux', 'Lille', 'Rennes', 'Reims', 'Saint-Étienne', 
 'Le Havre', 'Toulon', 'Grenoble', 'Dijon', 'Angers', 'Nîmes', 'Villeurbanne']

In [33]:
# Connexion à Snowflake
load_dotenv(r'D:\DATA\2025-11-28_MSPR-1_2\Good-Air\.env')

ACOUNT_SNOWFLAKE = os.getenv('ACOUNT_SNOWFLAKE')
USER_SNOWFLAKE = os.getenv('USER_SNOWFLAKE')
PASSWORD_SNOWFLAKE = os.getenv('PASSWORD_SNOWFLAKE')
conn = snowflake.connector.connect(
    user=USER_SNOWFLAKE,
    password=PASSWORD_SNOWFLAKE,
    account=ACOUNT_SNOWFLAKE,  # ex: "abcd-xy12345.europe-west4.gcp"
    warehouse="COMPUTE_WH",
    database="GOOD_AIR",
    schema="BRONZE"
)

In [6]:
#récupération de la derniére date de mise à jours de la table hubeau_clean
cur = conn.cursor()

cur.execute("""
    SELECT LAST_ALTERED
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME = 'BRONZE' 
      AND TABLE_SCHEMA = 'HUBEAU_SCRAP'
""")

result = cur.fetchone()
last_date_maj = ""

try :
    last_date_maj = result[0]
    last_date_maj = last_date_maj.replace(tzinfo=None)
    print(f" ************************* date snowflake récupéré {last_date_maj}")
except :
     print('Aucune date de mise a jours récente')

#A supp lors du déploiement
if not last_date_maj :
    last_date_maj = datetime.strptime("2020-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")

Aucune date de mise a jours récente


In [ ]:
#récupérer la liste des documents a téléchager

# Lancer le navigateur 
driver = webdriver.Chrome()

# Ouvre l'URL de ta page
driver.get("https://www.data.gouv.fr/datasets/resultats-du-controle-sanitaire-de-leau-distribuee-commune-par-commune/")

# Laisser un peu de temps pour charger la page
time.sleep(3)

# Récupérer tous les éléments avec la classe "space-y-2.5"
elements = driver.find_elements(By.CSS_SELECTOR, "div.border.border-gray-default.overflow-auto")

list_dis =[]

# Définir la locale en français
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Afficher leur texte
for  el in elements:

    print(el.text)
    dis = el.text
    if "dept" not in dis and ".pdf" not in dis :
        #extrait les dates de mis a jour des balise et les convertis en datetime
        date_maj = dis.split("\n")[1].split(" ")[4:7]
        date_maj ="/".join(date_maj)
        date_maj = datetime.strptime(date_maj, "%d/%B/%Y")
        print('date maj :', date_maj)

    dis = el.text
    if "dept" not in dis and ".pdf" not in dis :
        #extrait les dates de mis a jour des balise et les convertis en datetime
        date_maj = dis.split("\n")[1].split(" ")[4:7]
        date_maj ="/".join(date_maj)
        
        if "û" in date_maj :
            date_maj = date_maj.replace("û","Ã»")

        date_maj = datetime.strptime(date_maj, "%d/%B/%Y")

        #vérifie si la date du document à scrapper et plus récente que celle en bdd
        if date_maj > last_date_maj :
            print(dis.split("\n")[0], " " ,date_maj)

            try:
                # trouver le bouton <a> à l'intérieur de la div
                button = el.find_element(By.CSS_SELECTOR, "a.matomo_download")
                
                # Option 1 : cliquer directement
                button.click()
                time.sleep(2)  # petit délai pour le téléchargement

                time.sleep(300)
                driver.quit()
                
            except Exception as e:
                print(f"Élément : pas de bouton ou erreur - {e}")
        else :
            print("Dataset déjà à jour")
            driver.quit()



dis-2025.zip
Mis à jour le 3 novembre 2025 zip (202,3 Mo)
4K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
date maj : 2025-11-03 00:00:00
dis-2025-dept.zip
Mis à jour le 3 novembre 2025 zip (202,5 Mo)
2K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
dis-2024.zip
Mis à jour le 28 mars 2025 zip (274,4 Mo)
1K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
date maj : 2025-03-28 00:00:00
dis-2024-dept.zip
Mis à jour le 29 mars 2025 zip (274,6 Mo)
2K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
dis-2023.zip
Mis à jour le 14 octobre 2024 zip (278,4 Mo)
2K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
date maj : 2024-10-14 00:00:00
dis-2023-dept.zip
Mis à jour le 14 octobre 2024 zip (278,5 Mo)
3K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
dis-2022-dept.zip
Mis à jour le 7 juillet 2023 zip (293,9 Mo)
909
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
dis-2022.zip
Mis à jour le 7 juillet 2023 zip (293,7 Mo)
1K
TÉLÉCHARGER LA LISTE AU FORMAT
ZIP
date maj : 2023-07-07 00:00:00
dis-2021-dept.zip
Mis à jour le 11 août 2023 zip (292,4 Mo)
1K
TÉLÉCHARGER LA LISTE AU FORMAT
Z

ValueError: time data '11/août/2023' does not match format '%d/%B/%Y'

In [10]:
# chemin vers le dossier download
download_path = r"C:\Users\Utilisateur\Downloads"

list_donwload = os.listdir(download_path)
list_df = []

for doc in list_donwload :
    if "dis-" in doc : 
        
        zip_path = f"{download_path}\{doc}"
        
        # ouvrir le ZIP
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # liste des fichiers contenus dans le ZIP
            for file_name in zip_ref.namelist():
                    if "PLV" in file_name :
                        with zip_ref.open(file_name) as f:
                            # adapte le séparateur selon ton fichier (',' ou '\t' ou ';')
                            df = pd.read_csv(f, sep=",")
                            list_df.append(df)

df_concat = pd.concat(list_df)

# Dictionnaire de renommage
rename_dict = {
    'dateprel' : 'date_plv',
    'nomcommuneprinc': 'city_name',
    'plvconformitereferencebact': 'conformite_ref_bact',
    'plvconformitereferencechim' : 'conformite_ref_chim',
    'plvconformitebacterio' : 'conformite_bact',
    'plvconformitechimique' : 'conformite_chim',
    'conclusionprel' :'conclusion_plv',
}

# Renommer les colonnes
df_concat = df_concat.rename(columns=rename_dict)

#reduire le dict au colonne désiré /renomé
df = df_concat[list(rename_dict.values())]

<string>:10: SyntaxWarning: invalid escape sequence '\{'
<>:10: SyntaxWarning: invalid escape sequence '\{'
<string>:10: SyntaxWarning: invalid escape sequence '\{'
<>:10: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\516815102.py:10: SyntaxWarning: invalid escape sequence '\{'
  zip_path = f"{download_path}\{doc}"
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",")
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",")
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, se

In [14]:
#clean city_name
def clean_cities(city) :
    if city == 'NIMES' :
         return 'NÎMES'
    if city == 'SAINT-ETIENNE' :
         return 'SAINT-ÉTIENNE'
    if city == 'HAVRE (LE)' :
         return 'LE HAVRE'
    else :
         return city

def corel_city(row_city)   :
     for city in cities :
          if row_city.lower() in city.lower() :
               return city
          

df["city_name"] = df["city_name"].apply(clean_cities)
df["city_name"] = df["city_name"].apply(corel_city)

# Filtrer le DataFrame
df_reduce = df[df["city_name"].isin(cities)]
df_reduce["city_name"] = df_reduce["city_name"].str.upper()
df_reduce.reset_index(drop=True)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\1062825867.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["city_name"] = df["city_name"].apply(clean_cities)
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\1062825867.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["city_name"] = df["city_name"].apply(corel_city)
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\1062825867.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,date_plv,city_name,conformite_ref_bact,conformite_ref_chim,conformite_bact,conformite_chim,conclusion_plv
0,2021-07-22,PARIS,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
1,2021-01-04,NICE,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
2,2021-01-18,NICE,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
3,2021-02-01,NICE,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
4,2021-02-15,NICE,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
...,...,...,...,...,...,...,...
170469,2025-07-29,TOULON,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
170470,2025-03-25,TOULOUSE,N,C,C,C,Eau conforme aux limites de qualité réglementa...
170471,2025-04-01,TOULOUSE,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
170472,2025-03-25,TOULOUSE,S,C,S,C,Eau d'alimentation conforme aux exigences de q...


In [27]:
#transformer les nom de colonne en majusucle pour Snowflake 
df_reduce.columns = [c.upper() for c in df.columns]

In [34]:
#récupérer les id des city depuis snowflake
query  = """ SELECT CITY_NAME, CITY_ID FROM GOOD_AIR.SILVER.DIM_CITY """

try:
    df_dim_city = pd.read_sql(query, conn)
    print(f"Données chargées : {df_dim_city.shape[0]} lignes.")
finally:
    conn.close() # Toujours fermer la connexion

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_16824\1026264495.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dim_city = pd.read_sql(query, conn)


Données chargées : 20 lignes.


In [35]:
df_merge = pd.merge(df_reduce, df_dim_city, how ="left", on="CITY_NAME") 
df_merge.drop(columns= ["CITY_NAME"], inplace = True)

In [37]:
df_merge.columns

Index(['DATE_PLV', 'CONFORMITE_REF_BACT', 'CONFORMITE_REF_CHIM',
       'CONFORMITE_BACT', 'CONFORMITE_CHIM', 'CONCLUSION_PLV', 'CITY_ID'],
      dtype='object')

In [38]:

# Charger le DataFrame vers une table (la table doit exister)
success, nchunks, nrows, _ = write_pandas(conn, df_merge, "FACT_HUBEAU_RECORDS")

print(f"Upload terminé : {success}, {nrows} lignes insérées.")

MissingDependencyError: Missing optional dependency: pandas